# QT3 AFM Lab Image Processing Software Demo

Build the Thorlabs Educational Atomic Force Microscope according to the provided manual. Understand the steps to align the laser beam onto the probe and engage the probe with the sample. 

##### Tips for aligning the laser
Find the base of the cantilever where it meets the chip. Align as best as possible along the axis perpendicular to the cantilever. Move along the cantilever, recentering the beam as it becomes misaligned, until the beam is well on the far half of the cantilever, but still a safe distance from the tip. Make small adjustments on both mounts so that the beam is circular and centered in the diffusion lens.
##### Tips for engaging the probe
When engaging the probe with a real sample (not the microstructure), it's a good idea to make sure the surrounding surface is a similar height. Once you are engaged with the sample at 25V, walk the stage a few ticks in both directions for the x and y axes, stopping to lower the stage if the voltage drops below 25V. The surrounding surface can go above 25V but should not go significantly lower than 25V.

#### Taking an AFM scan and saving the file
Once the probe properly is engaged with the sample, take scans in Constant Force mode and observe the effect of different scan parameters such as Strain Gauge and resolution. Once a scan is complete, save the data as both an image and a csv file to an appropriate folder using the following format: 
###### parameter1Value1_parameter2Value2_parameter3Value3 (eg. nameMyScan_res250_speed100_modeConstantForce_straingauge)
This format is necessary for the software to identify the parameters of the scan. A parameter should be added to the file name if it is different from the default value. The parameters and default values are as according to the attributes defined in the ScanData class. Make sure you record each parameter correctly and with the correct spelling. Recommended order is according to how each is listed in the class, but order does not affect the operation of the software. Capitalization is optional but helpful for reading. Parameters that are either on or off only need to be in the file name if they are on for the scan and need only the parameter and no value. The last four attributes are not used in naming the file. Scans should be placed in the same folder if the data will be compared to one another

#### Identify and image features
Take a scan of the supplied microstructure in Constant Force mode with Strain Gauge and with 100px resolution at 100pps. The resolution of this image is not important because this scan is only for locating features. Once you see the feature you would like to look at closer, stop the scan and use the Zoom feature to take a closer scan of the desired feature with 250px resolution. Use the shift key while dragging to make sure you are taking a square window. If you need to restart the zoom scan, you have to reselect the zoom window because the start coordinates of the scan are not saved. When saving the data for a zoom scan, make sure to record the width of the zoom window to the file name using the width parameter (width3.0 corresponds to a zoom window of 3 microns by 3 microns).

#### Using the software
Import the necessary libraries and functions from the scandata file. Get scan data from your folder.

In [ ]:
from matplotlib import pyplot as plt
import numpy as np

from scandata import get_scan_data_from_directory, get_step_volts_for_calibration

afmscans = get_scan_data_from_directory('FunScans', includes=['Spore'])

Calibrate the height by calculating the volts per nm factor. Make sure this value is reasonable.

In [ ]:
volts = []
for afmscan in afmscans:
    volts.append(afmscan.get_edge().data_slice)

step_voltage = get_step_volts_for_calibration(volts)

for afmscan in afmscans:
    afmscan.cf_calibrate = step_voltage / 114

print(afmscans[0].cf_calibrate)

Plot afm images and horizontal slices to see what the surface of the sample looks like. When looking at the color axis, it's important to remember that a smaller voltage means a larger height dispacement. The default location of slice for get_edge() is the middle of the image, but if you want to look at a different slice of the image, use the y-axis coordinates to choose the section of the image you want using get_edge(y_coord= ).

In [ ]:
scan = afmscans[4]
scan.plot_afm_image().get_edge().volt_to_height().tilt_correct().plot_edge()

Plot an edge of each file to locate the coordinates of the step. For each edge, record the top and bottom coordinates of the step using the graph. Lists of coordinates of maxima and minima are provided to make this easier. It is helpful to separate the coordinates into forward and backward scans.

In [ ]:
steps = []
back_steps = []

for afmscan in afmscans:
    print("Backward Scan: " + str(afmscan.backward))
    afmscan.get_edge().volt_to_height().find_features().plot_edge()
    plt.show()
    top = float(input("Top Coordinate: "))
    bottom = float(input("Bottom Coordinate: "))
    if afmscan.backward:
        back_steps.append((top, bottom))
    else:
        steps.append((top, bottom))

print('steps: ', steps, 'back_steps: ', back_steps, sep='\n')

Using the lists of tuples you created, plot the step width and rms noise for each scan against the scan speed.

In [ ]:
# steps = [(0.8704, 0.6868), (1.0584, 0.8496), (1.0888, 0.908), (0.9792, 0.7632), (0.8132, 0.6384), (0.8092, 0.6256),
#          (1.1456, 0.9664), (0.8496, 0.6336), (0.9984, 0.6784), (0.7104, 0.4352), (1.5408, 1.296), (1.2528, 1.0728),
#          (0.64, 0.4544)]
# back_steps = [(0.816, 0.6324), (0.9936, 0.7848), (0.9996, 0.8092), (0.8784, 0.6768), (0.7828, 0.5244), (0.8364, 0.6256),
#               (1.0688, 0.8512), (0.7344, 0.5328), (0.896, 0.5952), (0.5824, 0.3136), (1.5192, 1.3464), (1.224, 1.044),
#               (0.608, 0.4224)]

# empty lists for plotting
step_widths, back_step_widths, scan_speeds, back_scan_speeds, scan_rms, back_scan_rms, scan_pid, back_scan_pid \
    = [[] for _ in range(8)]
# create plots
fig = plt.figure()
ax1 = plt.subplot2grid(shape=(2, 2), loc=(0, 0), colspan=2)
ax2 = plt.subplot2grid(shape=(2, 2), loc=(1, 0))
ax3 = plt.subplot2grid(shape=(2, 2), loc=(1, 1))
axs = (ax1, ax2, ax3)
[ax.grid(True) for ax in axs]
# fill empty lists with data and plot the edge
for afmscan, step in zip(afmscans, [item for pair in zip(steps, back_steps) for item in pair]):
    afmscan.get_edge().volt_to_height().tilt_correct((0, step[1]))
    # backward scans
    if afmscan.backward:
        back_scan_speeds.append(afmscan.speed)
        back_step_widths.append(afmscan.get_step_width(step))
        back_scan_rms.append(afmscan.get_noise((0, step[1])))
        back_scan_pid.append(afmscan.pid[0])
    # forward scans
    else:
        scan_speeds.append(afmscan.speed)
        step_widths.append(afmscan.get_step_width(step))
        scan_rms.append(afmscan.get_noise((0, step[1])))
        scan_pid.append(afmscan.pid[0])
    # plot slice of edge
    ax1.plot(np.linspace(0, afmscan.width, afmscan.res), afmscan.denoise())

ax1.set_title('Edge Resolution')
ax1.set_xlabel('x [microns]')
ax1.set_ylabel('height [nm]')

# plot step width against scan speed
[ax2.scatter(x, y, label=direction) for x, y, direction in
 zip([scan_speeds, back_scan_speeds], [step_widths, back_step_widths], ['forward', 'backward'])]
ax2.set_title('Step Widths')
ax2.set_xlabel('scan speeds [pps]')
ax2.set_ylabel('step width [microns]')
ax2.legend()
# plot rms noise against scan speed
[ax3.scatter(x, y, label=direction) for x, y, direction in
 zip([scan_speeds, back_scan_speeds], [scan_rms, back_scan_rms], ['forward', 'backward'])]
ax3.set_title('RMS Noise')
ax3.set_xlabel('scan speeds [pps]')
ax3.set_ylabel('rms [nm]')
ax3.legend()

plt.show()

Observe any patterns in step width and rms noise as the scan speed changes